In [1]:
# import neccessary packages
import numpy as np
import pandas as pd

In [2]:
########### DEPRIVATION
########### CREATE DEPRIVATION DATASET
# Read the data on deprivation for different years
df_deprivation = pd.DataFrame()
for year in range(1895, 1904):
    append_deprivation = pd.read_excel("Data_raw/Deprivation_Rights_MinJust_1894-1903.xlsx", 
                                           sheet_name=str(year)).assign(year_deprivation=str(year))
    df_deprivation = pd.concat([df_deprivation, append_deprivation], 
                       ignore_index=True)
for year in set(range(1904, 1913)).difference(set(range(1910, 1912))):
    append_deprivation = pd.read_excel("Data_raw/Deprivation_Rights_MinJust_1904-1914.xlsx", 
                                       sheet_name=str(year)).assign(year_deprivation=str(year))
    df_deprivation = pd.concat([df_deprivation, append_deprivation], 
                       ignore_index=True)
    
# Fix some typos in the courtnames and add cities
deprivation_fix_courtnames = pd.read_excel("auxiliary data/deprivation_fix_courtnames.xlsx")
df_deprivation = df_deprivation.merge(deprivation_fix_courtnames, left_on='Каким судебным учреждением', 
                                      right_on='court_old', how='left').drop(columns='court_old')
df_deprivation = df_deprivation.drop(columns='Каким судебным учреждением')

# Drop duplicates
df_deprivation = df_deprivation.drop_duplicates()

# Export data
df_deprivation.to_excel("cleaned_data/deprivation_raw.xlsx", index=False)

In [3]:
########### DEPRIVATION
########### GEOLOCATE
# Done after manual check for duplicates
# Read the data
df_location = pd.read_stata("auxiliary data/Administrative Structure and Coordinates_3.dta")
# Change city names to improve the geolocation
df_deprivation['city'].replace({'Елисаветполь' : 'Елизаветполь', 'Кутаиси' : 'Кутаис', 'Ташкент' : 'Ташкентское',
                               'Самарканд' : 'Самаркандское', 'Могилёв' : 'Могилев', 'Кишинёв' : 'Кишинев',
                               'Орёл' : 'Орел', 'Уральск' : 'Уральское', 'Семипалатинск' : 'Семипалатинское',
                               'Новый Маргелан' : 'Маргеланское'}, inplace = True)
# Merge the deprivation data with locations
df_deprivation_geolocated = pd.merge(df_deprivation, df_location, left_on='city', right_on='volost_new', how='left')
# Fix geolocation errors
df_deprivation_geolocated = df_deprivation_geolocated[~df_deprivation_geolocated['vnr'].isin([8310, 15319, 12663, 16125, 17325])]
df_deprivation_geolocated = df_deprivation_geolocated[df_deprivation_geolocated['villagename'] != "ст. Усть-Медведицкая"]

# Count the number of unsuccessful matches
print(len(df_deprivation_geolocated[df_deprivation_geolocated['gnr'].isna()]))
# Export the data
df_deprivation_geolocated.to_excel("cleaned_data/deprivation_geolocated.xlsx", index=False)

11


In [4]:
########### DEPRIVATION
########### COLLAPSE BY PROVINCE-YEAR
collapsed_deprivation = df_deprivation_geolocated[['Фамилия','gnr', 
                           'gub_new', 'year_deprivation']].groupby(['gnr', 'gub_new', 'year_deprivation']).count().unstack(fill_value=0).stack()
# Fix index and column names
collapsed_deprivation = collapsed_deprivation.reset_index()
collapsed_deprivation = collapsed_deprivation.rename(columns={'Фамилия': 'count_deprivation'})
# Export the data
collapsed_deprivation.to_excel("cleaned_data/deprivation_collapsed.xlsx", index=False)

In [5]:
########### OKHRANA
########### CREATE OKHRANA DATASET
# (1) Read the data on accused from Obzor/Vedomosti for different years 
df_accused_part1 = pd.read_excel("Data_raw/3) list of the accused.xlsx", 
                                 sheet_name='1895').drop(columns='№').assign(data_source='accused', years='1895')
df_accused_part2 = pd.read_excel("Data_raw/3) list of the accused.xlsx", 
                                 sheet_name='1895-1896').drop(columns='№').assign(data_source='accused', years='1895-1896')
df_accused_part3 = pd.read_excel("Data_raw/3) list of the accused.xlsx", 
                                 sheet_name='1898-1899').assign(data_source='accused', years='1898-1899')
# Fix NA values
df_accused_part3['Gendarme custody'] = 'С.-Петербургское'
df_accused_part4 = pd.read_excel("Data_raw/list of the accused 1900-1901.xlsx", 
                                 sheet_name='1900', names=['Gendarme custody', 
        'Surname', 'Name', 'Patronymic', 'Age', 'Religion', 'Ethnic group', 'title/rank', 'occupation', 
        'education', 'details', 'crime commited']).assign(data_source='accused', years='1900')
# Fix NA values
df_accused_part4['Gendarme custody'].fillna('Варшавское', inplace=True)
df_accused_part5 = pd.read_excel("Data_raw/list of the accused 1900-1901.xlsx", 
                                 sheet_name='1901', names=['Gendarme custody', 
        'Surname', 'Name', 'Patronymic', 'Age', 'Religion', 'Ethnic group', 'title/rank', 'occupation', 
        'education', 'details', 'crime commited']).assign(data_source='accused', years='1901')
# Append the data
df_accused = pd.concat([df_accused_part1, df_accused_part2, df_accused_part3, df_accused_part4, df_accused_part5], 
                       ignore_index=True)
# Keep only when name is available
df_accused = df_accused[df_accused['Name'].notna()]
# Drop empty column
df_accused.drop(['Unnamed: 13'], axis=1, inplace=True)

# (2) Read the data on lists revolutionaries accused of crimes against the state from Obzor/Vedomosti 
df_revolutionaries = pd.DataFrame()
for year in range(1887, 1898):
    append_revolutionaries = pd.read_excel("Data_raw/4) Okhrana_Social Conflict_Revolutionaries_Vedomost done.xlsx", 
                                           sheet_name=str(year)).assign(data_source='revolutionaries', years=str(year))
    df_revolutionaries = pd.concat([df_revolutionaries, append_revolutionaries], 
                       ignore_index=True)
# Fix column names
df_revolutionaries.rename({'Patronymic ' : 'Patronymic', 'Details' : 'details'}, inplace=True, axis=1)
# Clean the data
df_revolutionaries = df_revolutionaries[df_revolutionaries['Gendarme custody']!='В Оловецком и Омском Жандармском Управлениях дознаний произведено не было']
# Append to the Okhrana data set
df_okhrana = pd.concat([df_accused, df_revolutionaries], join='outer', ignore_index=True)

# COMMENTED PART. NOT USED
'''
# (2B) Append / merge names mentioned in Obzor
obzor_names = pd.read_excel("Data_raw/names mentioned in Obzor 1900-1901.xlsx", skiprows=[0])
obzor_names.columns = ['Surname', 'Name'] 
# Create a dummy indicator for names mentioned in Obzor
df_okhrana = pd.merge(df_okhrana, obzor_names, on=['Surname', 'Name'], how='outer', indicator=True)
# Create 'is_mentioned_obzor' column
df_okhrana['is_mentioned_obzor'] = (df_okhrana['_merge'] == 'both').astype(int)
# Fill years and data source
df_okhrana['years'].fillna('1900-1901', inplace=True)
df_okhrana['data_source'].fillna('mentioned_names', inplace=True)
# Drop the indicator column
df_okhrana = df_okhrana.drop('_merge', axis=1)
'''

# (3.1) Unify the names of the Gendarme custody
# Fix some typos in the courtnames and add cities
okhrana_fix_gendarme = pd.read_excel("auxiliary data/okhrana_fix_gendarme.xlsx")
df_okhrana = df_okhrana.merge(okhrana_fix_gendarme, left_on='Gendarme custody', 
                              right_on='gendarme_old', how='left').drop(columns='gendarme_old')

# (3.2) Preliminary drop dublicates using all the data from steps (1)-(2)
# Print number of observations
print(len(df_okhrana))
# Drop duplicates inside each of each data_source
df_okhrana = df_okhrana.drop_duplicates()
print(len(df_okhrana))
# More conservative definition of a duplicate (only for observations with surname)
df_okhrana_with_surname = df_okhrana[~df_okhrana['Surname'].isna()]
df_okhrana_with_surname = df_okhrana_with_surname.drop_duplicates(['Surname', 'Name', 'Patronymic', 'gendarme_new', 'years'])
df_okhrana_without_surname = df_okhrana[df_okhrana['Surname'].isna()]
df_okhrana = pd.concat([df_okhrana_with_surname, df_okhrana_without_surname], ignore_index=True)
print(len(df_okhrana))

7352
7344
7275


In [6]:
# (4) Append / merge People under investigation (4 files with names starting from Alphabet_People)
df_investigation_part1_1 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1886.xlsx", skiprows=[0], 
                        sheet_name='-178', names=["№", 'Surname', 'Name', 'Patronymic', 'title/rank', 'crime commited',
                                                  'Birthplace', 'Date of arrest or sentence', 'Age', 
                                                  'Distinguishing features', 'details', 
                                                  'Gendarme custody', 'Responsible official', 
                                                  'Comments']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1886', investigation_type='politically unreliable')
df_investigation_part1_2 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1886.xlsx", skiprows=[0], 
                        sheet_name='21-29', names=["№", 'Surname', 'Name', 'Patronymic', 'document', 
                                                   'Gendarme custody', 'Age', 'Birthplace', 
                                                   'title/rank']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1886', investigation_type='fled')
df_investigation_part1_3 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1886.xlsx", skiprows=[0], 
                        sheet_name='30', names=["№", 'Surname', 'Name', 'Patronymic', 'document', 
                                                   'Gendarme custody', 'Age', 'Birthplace', 
                                                   'title/rank']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1886', investigation_type='article 246')
df_investigation_part1_4 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1886.xlsx", skiprows=[0], 
                        sheet_name='31-33', names=["№", 'Surname', 'Name', 'Patronymic', 'title/rank',
                                                   'Age', 'crime commited', 'crime date', 
                                                   'instructions']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1886', investigation_type='non political')

df_investigation_part2_1 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1889+1893.xlsx", skiprows = [0],
                            sheet_name='1-338', names=["№", 'Surname', 'Name', 'Patronymic', 'title/rank', 'crime commited',
                                                  'Birthdate', 'Date of arrest or sentence', 'Age', 
                                                  'Gendarme custody', 'Responsible official', 
                                                  'Comments']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1889', investigation_type='police department')
df_investigation_part2_2 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1889+1893.xlsx", skiprows = [0],
                            sheet_name='341-342', names=["№", 'Surname', 'Name', 
                                                         'Patronymic', 'title/rank', 
                            'Gendarme custody', 
                            'Responsible official']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1889', investigation_type='article 246')
df_investigation_part2_3 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1889+1893.xlsx", skiprows = [0],
                            sheet_name='343-347', names=["№", 'Surname', 'Name', 'Patronymic', 'title/rank',
                            'crime commited',
                            'instructions']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1889', investigation_type='non political')
df_investigation_part2_4 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1889+1893.xlsx", skiprows = [0],
                        sheet_name='1-198', names=["№", 'Surname', 'Name', 'Patronymic','title/rank', 'crime commited',
                        'Birthplace', 'Date of arrest or sentence', 'Age', 
                        'Gendarme custody', 'Responsible official', 
                                                   'Comments']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1889', investigation_type='politically unreliable')
df_investigation_part2_5 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1889+1893.xlsx", skiprows = [0],
                        sheet_name='1-445', names=["№", 'Surname', 'Name', 'Patronymic',
                        'Birthdate', 'Birthplace', 'title/rank', 'crime commited',
                        'Date of arrest or sentence', 'Gendarme custody', 
                        'Responsible official', 'Comments']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1893', investigation_type='police department')
df_investigation_part2_6 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1889+1893.xlsx", skiprows = [0],
                        sheet_name='449-490', names=["№", 'Surname', 'Name', 'Patronymic',
                        'Birthdate', 'Birthplace', 'title/rank', 'crime commited',
                        'Date of arrest or sentence', 'Comments']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1893', investigation_type='measures 1889')
df_investigation_part2_7 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1889+1893.xlsx", skiprows = [0],
                        sheet_name='491-525', names=["№", 'Surname', 'Name', 'Patronymic',
                        'Birthdate', 'Birthplace', 'title/rank', 'crime commited',
                        'Date of arrest or sentence','Gendarme custody', 
                        'Responsible official', 'Comments']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1893', investigation_type='non political')
df_investigation_part3_1 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1899.xlsx", sheet_name='1', 
                                         skiprows=[0],
             names=["№", 'page', 'Surname', 'Name', 'Patronymic', 'title/rank',
                    'Date of arrest', 'details']).drop(columns=['№', 'page']).assign(data_source='investigation', 
                                                   years='1899', investigation_type='arrest')
df_investigation_part3_2 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1899.xlsx", sheet_name='1-820', 
                                         skiprows=[0],
             names=["№", 'Surname', 'Name', 'Patronymic', 'Birthdate', 'Birthplace', 'title/rank', 'crime commited',
                    'Date of arrest or sentence', 'Gendarme custody', 'Responsible official', 
                    'Comments']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1899', investigation_type='police department')
df_investigation_part3_3 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1899.xlsx", sheet_name='1-820', 
                                         skiprows=[0],
             names=["№", 'Surname', 'Name', 'Patronymic', 'Birthdate', 'Birthplace', 'title/rank', 
                    'crime commited', 'Date of arrest or sentence', 'Gendarme custody', 'Responsible official', 
                    'Comments']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1899', investigation_type='measures 1889')
df_investigation_part3_4 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1899.xlsx", skiprows=[0], 
                        sheet_name='851-905', names=["№", 'Surname', 'Name', 'Patronymic', 'Birthdate', 'title/rank',
                        'crime commited', 'ruling date', 'instructions',
                        'instructions_2', 'Gendarme custody']).drop(columns='№').assign(data_source='investigation', 
                                                   years='1899', investigation_type='non political')

df_investigation_part4 = pd.read_excel("Data_raw/Alphabet_People Under Investigation_1900.xlsx", skiprows=[0], 
                        names=['Surname', 'Name', 'Patronymic', 'title/rank', 
                               'document', 'Birthyear', 'instructions', 'instructions_2',
                              'instructions_3']).assign(data_source='investigation', 
                                                   years='1900', investigation_type='all')

df_investigation = pd.concat([df_investigation_part1_1, df_investigation_part1_2, df_investigation_part1_3, 
                        df_investigation_part1_4, df_investigation_part2_1, df_investigation_part2_2,
                        df_investigation_part2_3, df_investigation_part2_4,
                        df_investigation_part2_5, df_investigation_part2_6, df_investigation_part2_7,
                        df_investigation_part3_1, df_investigation_part3_2,
                       df_investigation_part3_3, df_investigation_part3_4, df_investigation_part4], 
                       ignore_index=True)

# (5.1) Unify the names of the Gendarme custody
# Fix some typos in the courtnames and add cities
investigation_fix_gendarme = pd.read_excel("auxiliary data/fix_gendarme_custody_investigation.xlsx")
df_investigation = df_investigation.merge(investigation_fix_gendarme, left_on='Gendarme custody', 
                              right_on='gendarme_old', how='left').drop(columns='gendarme_old')
# (5.2) Drop duplicates from the investigation part
# Print number of observations
print(len(df_investigation))
# Drop full duplicates
df_investigation = df_investigation.drop_duplicates()
print(len(df_investigation))
# More conservative definition of a duplicate (only for observations with surname)
df_investigation_with_surname = df_investigation[~df_investigation['Surname'].isna()]
df_investigation_with_surname = df_investigation_with_surname.drop_duplicates(['Surname', 
                                            'Name', 'Patronymic', 'gendarme_new', 'years'])
df_investigation_without_surname = df_investigation[df_investigation['Surname'].isna()]
df_investigation = pd.concat([df_investigation_with_surname, df_investigation_without_surname], ignore_index=True)
print(len(df_investigation))

7529
7524
6754


In [7]:
# (6) Merge two parts of the Okhrana data
df_okhrana_large = pd.concat([df_okhrana, df_investigation], join='outer', ignore_index=True)
# Print number of observations
print(len(df_okhrana_large))

# (7.1) Fix years
df_okhrana_large['years'].replace({'1895-1896' : '1896',
                               '1898-1899' : '1899'}, inplace = True)
df_okhrana_large.rename(columns={"years": "year_okhrana"}, inplace=True)

# (7.2) Additional search for duplicates
# account for capital VS lowercase letters
df_okhrana_large_without_surname = df_okhrana_large[df_okhrana_large['Surname'].isna()]
df_okhrana_large_with_surname = df_okhrana_large[~df_okhrana_large['Surname'].isna()]
df_okhrana_large_with_surname['surname'] = df_okhrana_large_with_surname['Surname'].apply(lambda x: str(x).lower())
df_okhrana_large_with_surname['name'] = df_okhrana_large_with_surname['Name'].apply(lambda x: str(x).lower())
df_okhrana_large_with_surname['patronymic'] = df_okhrana_large_with_surname['Patronymic'].apply(lambda x: str(x).lower())
df_okhrana_large_with_surname = df_okhrana_large_with_surname.drop_duplicates(subset=['surname', 
                                                    'name', 'patronymic', 'gendarme_new', 'year_okhrana'])
# Print number of observations
df_okhrana_large = pd.concat([df_okhrana_large_without_surname, df_okhrana_large_with_surname], join='outer', ignore_index=True)
print(len(df_okhrana_large))
# account for missing patronymic case
df_okhrana_large_without_patr = df_okhrana_large_with_surname[df_okhrana_large_with_surname['Patronymic'].isna()]
df_okhrana_large_with_patr = df_okhrana_large_with_surname[~df_okhrana_large_with_surname['Patronymic'].isna()]
# Create a dummy indicator for names existing also with patronymic
df_okhrana_large_without_patr = pd.merge(df_okhrana_large_without_patr, 
                                df_okhrana_large_with_patr, on=['surname', 'name',
                                                               'gendarme_new', 'year_okhrana'], how='left', 
                                         suffixes = ['', '_drop'], indicator=True)
# Drop unnecessary columns
df_okhrana_large_without_patr = df_okhrana_large_without_patr[df_okhrana_large_without_patr['_merge']=='left_only']
df_okhrana_large_without_patr = df_okhrana_large_without_patr[df_okhrana_large_without_patr.columns.drop(list(df_okhrana_large_without_patr.filter(regex='_drop')))]
# Print number of observations
df_okhrana_large = pd.concat([df_okhrana_large_without_surname, df_okhrana_large_without_patr,
                             df_okhrana_large_with_patr], join='outer', ignore_index=True)
df_okhrana_large.drop(columns=['surname', 'name', 'patronymic', '_merge'], inplace=True)
print(len(df_okhrana_large))

14029
14000
13956


C:\Users\aalip\AppData\Local\Temp\ipykernel_37208\4146798880.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_okhrana_large_with_surname['surname'] = df_okhrana_large_with_surname['Surname'].apply(lambda x: str(x).lower())
C:\Users\aalip\AppData\Local\Temp\ipykernel_37208\4146798880.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_okhrana_large_with_surname['name'] = df_okhrana_large_with_surname['Name'].apply(lambda x: str(x).lower())
C:\Users\aalip\AppData\Local\Temp\ipykernel_37208\41467988

In [8]:
# (8) Geolocate Okhrana data
# Read the data
df_location = pd.read_stata("auxiliary data/Administrative Structure and Coordinates_3.dta")
# Change city names to improve the geolocation
df_okhrana_large['city'].replace({'Елисаветполь' : 'Елизаветполь', 'Кутаиси' : 'Кутаис', 'Ташкент' : 'Ташкентское',
                               'Самарканд' : 'Самаркандское', 'Могилёв' : 'Могилев', 'Кишинёв' : 'Кишинев',
                               'Орёл' : 'Орел', 'Уральск' : 'Уральское', 'Семипалатинск' : 'Семипалатинское',
                               'Новый Маргелан' : 'Маргеланское'}, inplace = True)
# Merge the deprivation data with locations
df_okhrana_geolocated = pd.merge(df_okhrana_large, df_location, left_on='city', right_on='volost_new', how='left')
# Fix geolocation errors
df_okhrana_geolocated = df_okhrana_geolocated[~df_okhrana_geolocated['vnr'].isin([15319, 12663, 16125, 16918, 
                                                                                                17325])]
df_okhrana_geolocated = df_okhrana_geolocated[df_okhrana_geolocated['villagename'] != "ст. Усть-Медведицкая"]

# Count the number of unsuccessful matches
print(len(df_okhrana_geolocated[df_okhrana_geolocated['gnr'].isna()]))
# Export the data
df_okhrana_geolocated.to_excel("cleaned_data/okhrana_geolocated.xlsx", index=False)

5219


In [13]:
########### OKHRANA
########### COLLAPSE BY PROVINCE-YEAR
df_okhr_for_collapse = df_okhrana_geolocated[~df_okhrana_geolocated['gnr'].isna()]
df_okhr_for_collapse = df_okhr_for_collapse.assign(count_okhrana=1)
collapsed_okhrana = df_okhr_for_collapse[['count_okhrana','gnr', 
                           'gub_new', 'year_okhrana']].groupby(['gnr', 'gub_new', 'year_okhrana']).count().unstack(fill_value=0).stack()
# Fix index 
collapsed_okhrana = collapsed_okhrana.reset_index()
# Export the data
collapsed_okhrana.to_excel("cleaned_data/okhrana_collapsed.xlsx", index=False)